In [1]:
import glob
import sys
import time
import os.path
import copy
import numpy as np
import healpy
import fitsio
from multiprocessing import Pool

#from .utils import userprint

import glob
from matplotlib import pyplot as plt 
from astropy.io import fits
from astropy.table import Table, join, vstack


In [2]:
def userprint(*args, **kwds):
    """Defines an extension of the print function.
    Args:
        *args: arguments passed to print
        **kwargs: keyword arguments passed to print
    """
    print(*args, **kwds)
    sys.stdout.flush()

    
#line 1439 in master/py/picca/io.py
 
    
def read_deltas(in_dir,
                nside,
                lambda_abs,
                alpha,
                z_ref,
                cosmo,
                max_num_spec=None,
                no_project=False,
                from_image=None,
                nproc=None):
    
    files = []
    in_dir = os.path.expandvars(in_dir)
    if from_image is None or len(from_image) == 0:
        if len(in_dir) > 8 and in_dir[-8:] == '.fits.gz':
            files += sorted(glob.glob(in_dir))
        elif len(in_dir) > 5 and in_dir[-5:] == '.fits':
            files += sorted(glob.glob(in_dir))
        else:
            files += sorted(glob.glob(in_dir + '/*.fits') + glob.glob(in_dir +
                                                               '/*.fits.gz'))
    else:
        for arg in from_image:
            if len(arg) > 8 and arg[-8:] == '.fits.gz':
                files += sorted(glob.glob(arg))
            elif len(arg) > 5 and arg[-5:] == '.fits':
                files += sorted(glob.glob(arg))
            else:
                files += sorted(glob.glob(arg + '/*.fits') + glob.glob(arg +
                                                                '/*.fits.gz'))
    files = sorted(files)

    arguments = [(f, from_image) for f in files]
    pool = Pool(processes=nproc)
    results = pool.starmap(read_delta_file, arguments)
    pool.close()

    deltas = []
    num_data = 0
    for delta in results:
        if delta is not None:
            deltas += delta
            num_data = len(deltas)
            if (max_num_spec is not None) and (num_data > max_num_spec):
                break

    # truncate the deltas if we load too many lines of sight
    if max_num_spec is not None:
        deltas = deltas[:max_num_spec]
        num_data = len(deltas)

    userprint("\n")
    
    
    # compute healpix numbers
    phi = [delta.ra for delta in deltas]
    theta = [np.pi / 2. - delta.dec for delta in deltas]
    healpixs = healpy.ang2pix(nside, theta, phi)
    if healpixs.size == 0:
        raise AssertionError('ERROR: No data in {}'.format(in_dir))

    data = {}
    z_min = 10**deltas[0].log_lambda[0] / lambda_abs - 1.
    z_max = 0.
    for delta, healpix in zip(deltas, healpixs):
        z = 10**delta.log_lambda / lambda_abs - 1.
        z_min = min(z_min, z.min())
        z_max = max(z_max, z.max())
        delta.z = z
        if not cosmo is None:
            delta.r_comov = cosmo.get_r_comov(z)
            delta.dist_m = cosmo.get_dist_m(z)
        delta.weights *= ((1 + z) / (1 + z_ref))**(alpha - 1)

        if not no_project:
            delta.project()

        if not healpix in data:
            data[healpix] = []
        data[healpix].append(delta)

    return data, num_data, z_min, z_max

In [3]:
filename='/global/homes/s/sgontcho/desi/desi-everest/Delta_LYA/delta-1.fits.gz' 
l_ref=1216
hdul = fits.open(filename)
Deltas=np.array([])
Redshifts=np.array([])
RAs=np.array([])
DECs=np.array([])
datalist=[]
for m in range(0,173):
    
    data = Table(hdul[m].data)
    for row in range(len(data)):
        Deltas=np.append(Deltas,data.field(1)[row])
        RAs=np.append(RAs,hdul[m].header['RA'])
        DECs=np.append(DECs,hdul[m].header['DEC'])
        loglam=data.field(0)[row]
        l_obs=10**(loglam)
        z=(l_obs-l_ref)/l_ref
        Redshifts=np.append(Redshifts,z)
        

for m in range (len(Deltas)-1):
    datalist.append([RAs[m],DECs[m],Redshifts[m],Deltas[m]])                               
                   


In [4]:
deltatable = np.array(datalist)

In [5]:
deltatable

array([[ 2.4381757 ,  1.42547365,  1.96052632,  4.35542124],
       [ 2.4381757 ,  1.42547365,  1.96257208,  1.66310591],
       [ 2.4381757 ,  1.42547365,  1.96461926,  1.2053988 ],
       ...,
       [ 2.35698501,  1.47303206,  2.42270072, -0.60151686],
       [ 2.35698501,  1.47303206,  2.42506585,  0.04354024],
       [ 2.35698501,  1.47303206,  2.42743262,  0.31504151]])

In [6]:
col1 = fits.Column(name='ra', format='E', array=RAs)
col2 = fits.Column(name='dec', format='E', array=DECs)
col3 = fits.Column(name='z', format='E', array=Redshifts)
col4 = fits.Column(name='Delta', format='E', array=Deltas)

cols=fits.ColDefs([col1,col2,col3,col4])
hdu = fits.BinTableHDU.from_columns(cols)
hdu.writeto('/global/homes/s/schampat/Voids/VAST/VoidFinder/deltatable.fits',overwrite=True)


In [7]:
filename='/global/homes/s/schampat/Voids/VAST/VoidFinder/deltatable.fits'
hdul = fits.open(filename)
data = Table(hdul[1].data)
data




ra,dec,z,Delta
float32,float32,float32,float32
2.4381757,1.4254737,1.9605263,4.355421
2.4381757,1.4254737,1.9625721,1.663106
2.4381757,1.4254737,1.9646193,1.2053988
2.4381757,1.4254737,1.9666679,1.8245617
2.4381757,1.4254737,1.9687178,-1.0706358
2.4381757,1.4254737,1.9707693,1.4216183
2.4381757,1.4254737,1.9728222,0.71694505
2.4381757,1.4254737,1.9748764,-0.36280432
2.4381757,1.4254737,1.976932,-0.7164737


In [8]:
hdu = fits.open('/global/homes/s/schampat/Voids/VAST/VoidFinder/deltatable.fits')

data_table = Table(hdu[1].data)

print(data_table)

    ra       dec        z        Delta    
--------- --------- --------- ------------
2.4381757 1.4254737 1.9605263     4.355421
2.4381757 1.4254737 1.9625721     1.663106
2.4381757 1.4254737 1.9646193    1.2053988
2.4381757 1.4254737 1.9666679    1.8245617
2.4381757 1.4254737 1.9687178   -1.0706358
2.4381757 1.4254737 1.9707693    1.4216183
2.4381757 1.4254737 1.9728222   0.71694505
2.4381757 1.4254737 1.9748764  -0.36280432
2.4381757 1.4254737  1.976932   -0.7164737
2.4381757 1.4254737 1.9789891   0.50402004
      ...       ...       ...          ...
 2.356985 1.4730321 2.4061904   0.14506601
 2.356985 1.4730321  2.408544      0.47213
 2.356985 1.4730321 2.4108994   0.11319674
 2.356985 1.4730321 2.4132564   0.49564514
 2.356985 1.4730321  2.415615  -0.45444122
 2.356985 1.4730321 2.4179754 -0.040889047
 2.356985 1.4730321 2.4203372    1.2626777
 2.356985 1.4730321 2.4227006  -0.60151684
 2.356985 1.4730321 2.4250658   0.04354024
 2.356985 1.4730321 2.4274325    0.3150415
 2.356985 1